## Install transformers

In [1]:
# install dependencies
!pip install transformers

     |████████████████████████████████| 778kB 8.5MB/s 
     |████████████████████████████████| 1.1MB 44.0MB/s 
     |████████████████████████████████| 3.0MB 57.9MB/s 
     |████████████████████████████████| 890kB 51.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=693e32825ec5f077eceb5e9c35a68403bbc7cf3bdfd39b21c7c206af18e309c9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


## Initialize the pipeline QA model

In [1]:
# load libraries
from transformers import pipeline

# initialize question-answering task
nlp = pipeline('question-answering')

### Test the model

In [16]:
context = "Taylor Swift and her team are taking swift action after they were accused of copying the logo of a Black-owned business on merchandise for the album folklore. \
Amira Rasool, founder of the online retailer The Folklore, accused the pop star last week of selling merchandise that ripped off the logo of her company, \
which sells apparel, accessories and other products by designers in Africa and the diaspora. \
Rasool shared photos on Twitter and Instagram that showed cardigans and sweatshirts with the words The Folklore Album for sale on Swift's website."

question = "Who is accused?"

result = nlp({"question": question, "context": context})
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: 'Taylor Swift', score: 0.8689, start: 0, end: 12


## Initialize AutoTokenizer and TFAutoModelForQuestionAnswering 

In [13]:
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = TFAutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

All model checkpoint weights were used when initializing TFBertForQuestionAnswering.

All the weights of TFBertForQuestionAnswering were initialized from the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


### Test the model

In [14]:
text = "Taylor Swift and her team are taking swift action after they were accused of copying the logo of a Black-owned business on merchandise for the album folklore. \
Amira Rasool, founder of the online retailer The Folklore, accused the pop star last week of selling merchandise that ripped off the logo of her company, \
which sells apparel, accessories and other products by designers in Africa and the diaspora. \
Rasool shared photos on Twitter and Instagram that showed cardigans and sweatshirts with the words The Folklore Album for sale on Swift's website."

questions = ["Who is accused?",
              "What is the album name?",
              "Why is she accused?" ]

for question in questions:
  inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="tf")
  input_ids = inputs["input_ids"].numpy()[0]

  text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
  answer_start_scores, answer_end_scores = model(inputs)

  answer_start = tf.argmax(answer_start_scores, axis=1).numpy()[0]  # Get the most likely beginning of answer with the argmax of the score
  answer_end = (tf.argmax(answer_end_scores, axis=1) + 1).numpy()[0]  # Get the most likely end of answer with the argmax of the score
  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

  print(f"Question: {question}")
  print(f"Answer: {answer}")

Question: Who is accused?
Answer: amira rasool
Question: What is the album name?
Answer: folklore
Question: Why is she accused?
Answer: selling merchandise that ripped off the logo of her company


Reference: https://huggingface.co/transformers/task_summary.html